In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

import warnings

import seaborn as sns
import matplotlib.pyplot as plt

import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime

import warnings
from arch.compat.numba import jit
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("bmh")
plt.rcParams['font.family'] = 'DejaVu Sans'

import scipy.stats as ss
from arch import arch_model
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.style.use("bmh")

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
def raw_adj_close_prices(ticker: str, date_from: str, date_to: str):
    prices = yf.download(ticker, date_from, date_to)
    prices.index = prices.index.to_period(freq='d')
    return prices['Adj Close']

In [ ]:
def raw_adj_close_log_returns(prices: pd.Series):
    return np.log(prices).diff().dropna()

In [ ]:
def beta(market: pd.Series, single_stock: pd.Series, lag: int = 252):
    if True:
        return 0 # fixme
    return market.cov(single_stock, lag) / market.std(ddof=lag)

In [ ]:
@jit
def arch_filtered_series(returns: pd.Series,
                         dist: str = 'Normal',
                         mean: str = 'HARX',
                         vol: str = 'Garch',
                         lag: int = 200,
                         p: int = 1,
                         o: int = 0,
                         q: int = 1,
                         forecast_horizon: int = 30):
    scaling_const = 10.0 / returns.std()

    model = arch_model(scaling_const * returns,
                       mean=mean, lags=lag, # mean = Constant, ARX, HARX + the number of lags
                       vol=vol, p=p, o=o, q=q, # vol = Garch, EGARCH, HARCH + the number of lags
                       dist=dist) # dist = Normal, t, skewstudent, ged

    res = model.fit(update_freq=0, disp='off')
    stand_residuals = res.resid / res.conditional_volatility
    forecast = res.forecast(horizon=forecast_horizon)
    means = pd.Series(forecast.mean.tail(1).to_numpy()[0])
    vars = pd.Series(forecast.variance.tail(1).to_numpy()[0])
    return stand_residuals, means, vars

In [ ]:
DATE_FROM = '2015-01-01'
DATE_TO = '2017-12-31'
tickers = ['TSLA', 'AAPL', 'IBM', 'AMZN', 'MMM',
           'ABMD', 'ACN', 'APD', 'GOOGL', 'BLK']

In [ ]:
BETA_LAG = 252
GARCH_LAG = 200
CORRELATION_MATRIX_LAG = 200
GARCH_REFIT_DELAY_DAYS = 30

In [ ]:
spy_p = raw_adj_close_prices('SPY', DATE_FROM, DATE_TO)

raw_prices = []
for i in range(0, len(tickers)):
    raw_prices.append(raw_adj_close_prices(tickers[i], DATE_FROM, DATE_TO))


In [ ]:
plt.figure(figsize=(16,7))
(spy_p / spy_p.mean()).plot()
for data in raw_prices:
    (data / data.mean()).plot()
plt.title("normalized prices")
plt.legend(tickers)
plt.show()

In [ ]:
spy = raw_adj_close_log_returns(spy_p)
plt.figure(figsize=(16,7))
spy.plot()
plt.title("spy log returns")
plt.show()

In [ ]:
spyf, spy_means, spy_vars = arch_filtered_series(spy)
plt.figure(figsize=(16,7))
spyf.plot()
plt.title("spy filtered log returns")
plt.show()

In [ ]:
spy_means.plot()
plt.show()

In [ ]:
plt.figure(figsize=(16,7))
plot_acf(spy)
plt.show()


In [ ]:
plt.figure(figsize=(16,7))
plot_acf(spyf)
plt.show()

In [ ]:
res = spy_p.values
result = adfuller(res)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
res = spy.values
result = adfuller(res)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
res = spyf.dropna().values
result = adfuller(res)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

In [ ]:
# betas = {}
# for ticker, data in raw_prices.items():
#     betas[ticker] = beta(spy_p, data)
#
# betas

In [ ]:
# net_market = raw_adj_close_log_returns(raw_prices['TSLA']) - spy * betas['TSLA']
# plt.figure(figsize=(16,7))
# net_market.plot()
# plt.show()

In [ ]:
# teslaf, _ = arch_filtered_series(net_market)
# plt.figure(figsize=(16,7))
# teslaf.plot()
# plt.title("tsla filtered log returns")
# plt.show()

In [ ]:
# res = net_market.values
# result = adfuller(res)
# print('ADF Statistic: %f' % result[0])
# print('p-value: %f' % result[1])
# print('Critical Values:')
# for key, value in result[4].items():
#     print('\t%s: %.3f' % (key, value))
#
# res = teslaf.dropna().values
# result = adfuller(res)
# print('ADF Statistic: %f' % result[0])
# print('p-value: %f' % result[1])
# print('Critical Values:')
# for key, value in result[4].items():
#     print('\t%s: %.3f' % (key, value))
#

In [ ]:
series, means, vars = arch_filtered_series(spy)

In [ ]:
mkt_returns = []
for i in range(0, len(raw_prices)):
    ticker = tickers[i]
    data = raw_prices[i]
    mkt_returns.append(raw_adj_close_log_returns(data))

betas = []
for i in range(0, len(mkt_returns)):
    ticker = tickers[i]
    data = mkt_returns[i]
    betas.append(beta(spy, data))

net_market_returns = []
for i in range(0, len(mkt_returns)):
    ticker = tickers[i]
    data = mkt_returns[i]
    net_market_returns.append(data - spy * betas[i])

In [ ]:
net_market_returns_f = []
means = []
vars = []
for i in range(0, len(net_market_returns)):
    ticker = tickers[i]
    net_rets = net_market_returns[i]
    fltr_res, fcst_means, fcst_vars = arch_filtered_series(net_rets)
    net_market_returns_f.append(fltr_res)
    means.append(fcst_means.dropna())
    vars.append(fcst_vars.dropna())

net_market_returns_f


In [ ]:
returns = {}
for i in range(0, len(tickers)):
    returns[tickers[i]] = net_market_returns_f[i].dropna()

data = pd.DataFrame(returns, columns=tickers)

In [ ]:
data

In [ ]:
corr_matrix = data.corr(min_periods=200)
corr_matrix

In [ ]:
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
corr_matrix.iloc[0]


In [ ]:
def remove_row_and_column(matrix, index: int):
    tmp = np.delete(matrix, (index), axis=0)
    return np.delete(tmp, (index), axis=1)

In [ ]:
mus = [] ### fixme: move to indicies
for i in range(0, len(tickers)):
    r_j_j = remove_row_and_column(corr_matrix, i)
    inv_r_j_j = np.linalg.inv(r_j_j)
    mu_j = np.delete(means, i)
    sigma_j = np.delete(vars, i)
    v = (real_returns - mu_j) / sigma_j
    mus.append(means[i][0] + vars[i][0] * corr_matrix[i].dot(inv_r_j_j).dot(v))

In [ ]:
def alpha(mu, real_return):
    return 2 * mu - real_return